# Ironclad Face Recognition Demo

This notebook demonstrates the core functionality of the Ironclad face recognition system.

## Features Demonstrated:
1. Face embedding extraction using VGGFace2
2. Building a searchable index with FAISS HNSW
3. Face identification and similarity search
4. Performance comparison between different search methods

## Setup
Make sure you have all dependencies installed:
```bash
pip install torch torchvision pillow numpy faiss-cpu flask
```

In [ ]:
import os
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path

# Import ironclad modules
from ironclad.modules.extraction.embedding import Embedding
from ironclad.modules.extraction.preprocessing import Preprocessing
from ironclad.modules.retrieval.index.hnsw import FaissHNSW
from ironclad.modules.retrieval.index.bruteforce import BruteForce
from ironclad.modules.retrieval.search import FaissSearch

## 1. Setup and Configuration

In [ ]:
# Configuration
GALLERY_DIR = 'demo_data/gallery'
QUERY_DIR = 'demo_data/queries'
MODEL = 'vggface2'  # Options: 'vggface2', 'facenet'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Using device: {DEVICE}")
print(f"Model: {MODEL}")

## 2. Initialize Models

In [ ]:
# Initialize preprocessing and embedding model
preprocessor = Preprocessing()
embedding_model = Embedding(pretrained=MODEL, device=DEVICE)

print("Models initialized successfully!")

## 3. Load Gallery Images

Load and process all images from the gallery to build our face database.

In [ ]:
def load_gallery_images(gallery_path):
    """Load all images from gallery directory."""
    gallery_images = []
    gallery_labels = []
    
    for person_name in os.listdir(gallery_path):
        person_dir = os.path.join(gallery_path, person_name)
        if not os.path.isdir(person_dir):
            continue
            
        for img_file in os.listdir(person_dir):
            if img_file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(person_dir, img_file)
                gallery_images.append(img_path)
                gallery_labels.append(person_name)
    
    return gallery_images, gallery_labels

gallery_images, gallery_labels = load_gallery_images(GALLERY_DIR)
print(f"Loaded {len(gallery_images)} images from {len(set(gallery_labels))} people")
print(f"People in gallery: {sorted(set(gallery_labels))}")

## 4. Visualize Sample Gallery Images

In [ ]:
# Display a few sample images from the gallery
fig, axes = plt.subplots(1, min(5, len(gallery_images)), figsize=(15, 3))
if len(gallery_images) == 1:
    axes = [axes]

for i, (img_path, label) in enumerate(zip(gallery_images[:5], gallery_labels[:5])):
    img = Image.open(img_path)
    axes[i].imshow(img)
    axes[i].set_title(label.replace('_', ' '))
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 5. Extract Face Embeddings

Generate embedding vectors for all gallery images.

In [ ]:
def extract_embeddings(image_paths, preprocessor, model):
    """Extract embeddings for a list of images."""
    embeddings = []
    
    for img_path in image_paths:
        # Load and preprocess image
        img = Image.open(img_path)
        processed_img = preprocessor(img)
        
        # Extract embedding
        embedding = model(processed_img)
        embeddings.append(embedding)
    
    return np.array(embeddings)

print("Extracting gallery embeddings...")
gallery_embeddings = extract_embeddings(gallery_images, preprocessor, embedding_model)
print(f"Gallery embeddings shape: {gallery_embeddings.shape}")
print(f"Embedding dimension: {gallery_embeddings.shape[1]}")

## 6. Build Search Indices

Create both HNSW (fast approximate) and Brute-Force (exact) search indices.

In [ ]:
# Build HNSW index for fast approximate nearest neighbor search
print("Building HNSW index...")
hnsw_index = FaissHNSW(similarity_measure='cosine', m=16, ef_construction=200)
hnsw_index.build(gallery_embeddings)
print("HNSW index built!")

# Build brute-force index for exact search (comparison baseline)
print("\nBuilding Brute-Force index...")
bf_index = BruteForce(similarity_measure='cosine')
bf_index.build(gallery_embeddings)
print("Brute-Force index built!")

## 7. Face Identification Demo

Test face identification with query images.

In [ ]:
def identify_face(query_image_path, index, k=3):
    """Identify a face and return top-k matches."""
    # Load and preprocess query image
    img = Image.open(query_image_path)
    processed_img = preprocessor(img)
    
    # Extract embedding
    query_embedding = embedding_model(processed_img)
    
    # Search for similar faces
    distances, indices = index.search(query_embedding.reshape(1, -1), k=k)
    
    # Get top-k matches
    results = []
    for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
        results.append({
            'rank': i + 1,
            'name': gallery_labels[idx],
            'image': gallery_images[idx],
            'distance': dist
        })
    
    return img, results

# Get a query image
query_images = []
for person_name in os.listdir(QUERY_DIR):
    person_dir = os.path.join(QUERY_DIR, person_name)
    if os.path.isdir(person_dir):
        for img_file in os.listdir(person_dir):
            if img_file.endswith(('.jpg', '.jpeg', '.png')):
                query_images.append(os.path.join(person_dir, img_file))

if query_images:
    query_path = query_images[0]
    query_name = Path(query_path).parent.name
    
    print(f"Query image: {query_name}")
    query_img, results = identify_face(query_path, hnsw_index, k=3)
    
    # Visualize results
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    
    # Show query image
    axes[0].imshow(query_img)
    axes[0].set_title(f'Query: {query_name.replace("_", " ")}', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # Show top-3 matches
    for i, result in enumerate(results):
        match_img = Image.open(result['image'])
        axes[i+1].imshow(match_img)
        axes[i+1].set_title(
            f"#{result['rank']}: {result['name'].replace('_', ' ')}\nDist: {result['distance']:.3f}",
            fontsize=10
        )
        axes[i+1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\nTop-3 Matches:")
    for result in results:
        print(f"  {result['rank']}. {result['name']} (distance: {result['distance']:.4f})")
else:
    print("No query images found!")

## 8. Performance Comparison: HNSW vs Brute-Force

Compare speed and accuracy between approximate (HNSW) and exact (Brute-Force) search.

In [ ]:
import time

if query_images:
    # Prepare query embedding
    query_path = query_images[0]
    img = Image.open(query_path)
    processed_img = preprocessor(img)
    query_embedding = embedding_model(processed_img).reshape(1, -1)
    
    # Benchmark HNSW
    start = time.time()
    hnsw_dists, hnsw_indices = hnsw_index.search(query_embedding, k=5)
    hnsw_time = time.time() - start
    
    # Benchmark Brute-Force
    start = time.time()
    bf_dists, bf_indices = bf_index.search(query_embedding, k=5)
    bf_time = time.time() - start
    
    print("Performance Comparison:")
    print(f"  HNSW Search Time: {hnsw_time*1000:.2f} ms")
    print(f"  Brute-Force Search Time: {bf_time*1000:.2f} ms")
    print(f"  Speedup: {bf_time/hnsw_time:.2f}x")
    
    # Check if results match (they should be very similar)
    matching_results = np.sum(hnsw_indices[0] == bf_indices[0])
    print(f"\n  Top-5 Results Match: {matching_results}/5")
    print(f"  HNSW Top-1: {gallery_labels[hnsw_indices[0][0]]}")
    print(f"  BF Top-1: {gallery_labels[bf_indices[0][0]]}")

## 9. Embedding Space Visualization

Visualize the embedding space using dimensionality reduction.

In [ ]:
from sklearn.decomposition import PCA

# Reduce embeddings to 2D for visualization
pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(gallery_embeddings)

# Create color map for different people
unique_labels = sorted(set(gallery_labels))
label_to_color = {label: i for i, label in enumerate(unique_labels)}
colors = [label_to_color[label] for label in gallery_labels]

# Plot
plt.figure(figsize=(10, 8))
scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], 
                     c=colors, cmap='tab10', s=100, alpha=0.6)
plt.title('Face Embedding Space (PCA Projection)', fontsize=14, fontweight='bold')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')

# Add legend
handles = [plt.Line2D([0], [0], marker='o', color='w', 
                     markerfacecolor=scatter.cmap(scatter.norm(i)), 
                     markersize=8, label=label.replace('_', ' ')) 
          for i, label in enumerate(unique_labels)]
plt.legend(handles=handles, loc='best')

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Explained variance: {pca.explained_variance_ratio_.sum():.2%}")

## Summary

This demo showcased:
1. ✅ Face embedding extraction using state-of-the-art models
2. ✅ Fast similarity search with FAISS HNSW
3. ✅ Face identification with top-k matching
4. ✅ Performance comparison between search methods
5. ✅ Embedding space visualization

### Next Steps
- Explore the benchmarking scripts in the repository
- Try different embedding models (VGGFace2, FaceNet)
- Test with larger datasets
- Run the Flask API server for production use

### Key Takeaways
- **HNSW provides significant speedup** over brute-force search with minimal accuracy loss
- **Face embeddings cluster by identity** in the high-dimensional space
- **System is modular and extensible** for different models and indexing methods